In [1]:
import pandas as pd
from typing import List
import os

def combine_dataframes_with_unique_columns(csv_paths: List[str]) -> pd.DataFrame:
    """
    Combines multiple CSV files into a single DataFrame, only adding new unique columns.
    Existing columns are not duplicated.
    
    Args:
        csv_paths (List[str]): List of paths to CSV files
        
    Returns:
        pd.DataFrame: Combined DataFrame with unique columns
    """
    # Initialize an empty DataFrame for the result
    combined_df = pd.DataFrame()
    
    # Keep track of columns we've already added
    existing_columns = set()
    
    for file_path in csv_paths:
        try:
            # Read the current CSV file
            current_df = pd.read_csv(file_path)
            
            # Get new columns that don't exist in our combined DataFrame
            new_columns = [col for col in current_df.columns if col not in existing_columns]
            
            if new_columns:
                # If this is the first DataFrame, use it as is
                if combined_df.empty:
                    combined_df = current_df
                else:
                    # Only add the new columns to the combined DataFrame
                    combined_df = pd.concat([combined_df, current_df[new_columns]], axis=1)
                
                # Update our set of existing columns
                existing_columns.update(new_columns)
                
            print(f"Processed {os.path.basename(file_path)}: Added {len(new_columns)} new columns")
            
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")
            continue
    
    return combined_df

# Example usage:
csv_paths = [
    '/work/process/combined_top5_desk.csv',
    '/work/process/combined_top5_accept.csv',
    '/work/process/combined_top5_accept_rating.csv',
    '/work/process/combined_top5_accept_rating_criteria.csv',
    '/work/process/combined_top_conference.csv',
    '/work/process/combined_tenure_eval.csv',
    '/work/process/combined_research_award.csv',
    '/work/process/combined_nobel_potential.csv',
    '/work/process/combined_grant.csv',
    '/work/process/combined_citation_impact.csv',
    # Add more paths as needed
]

# Combine the DataFrames
result_df = combine_dataframes_with_unique_columns(csv_paths)

# Optional: Save the combined DataFrame to a new CSV
# result_df.to_csv('/work/process/final_combined_data.csv', index=False)

Processed combined_top5_desk.csv: Added 10 new columns
Processed combined_top5_accept.csv: Added 4 new columns
Processed combined_top5_accept_rating.csv: Added 4 new columns
Processed combined_top5_accept_rating_criteria.csv: Added 4 new columns
Processed combined_top_conference.csv: Added 4 new columns
Processed combined_tenure_eval.csv: Added 4 new columns
Processed combined_research_award.csv: Added 4 new columns
Processed combined_nobel_potential.csv: Added 4 new columns
Processed combined_grant.csv: Added 4 new columns
Processed combined_citation_impact.csv: Added 4 new columns


In [3]:
result_df

,Submission_id,Paper_id,Submission,Original_Publication,Name_Category,Institution,top5_desk_1,top5_desk_2,top5_desk_3,top5_desk_mean,...,nobel_potential_3,nobel_potential_mean,grant_1,grant_2,grant_3,grant_mean,citation_impact_1,citation_impact_2,citation_impact_3,citation_impact_mean
0,S_000001,1,A submission with the following details: Title...,Journal of Political Economy,Top,Massachusetts Institute of Technology;,10.0,9.0,10.0,9.67,...,8.0,8.67,9.0,9.0,8.0,8.67,55.0,150.0,78.0,94.33
1,S_000002,1,A submission with the following details: Title...,Journal of Political Economy,Top,Harvard University;,10.0,10.0,9.0,9.67,...,8.0,9.00,9.0,9.0,9.0,9.00,150.0,250.0,265.0,221.67
2,S_000003,1,A submission with the following details: Title...,Journal of Political Economy,Top,University of Warwick;,10.0,9.0,9.0,9.33,...,9.0,9.00,9.0,9.0,9.0,9.00,120.0,75.0,125.0,106.67
3,S_000004,1,A submission with the following details: Title...,Journal of Political Economy,Top,London School of Economics and Political Science;,9.0,9.0,10.0,9.33,...,8.0,8.33,9.0,10.0,9.0,9.33,85.0,100.0,120.0,101.67
4,S_000005,1,A submission with the following details: Title...,Journal of Political Economy,Top,University of Tokyo;,8.0,9.0,8.0,8.33,...,8.0,8.00,9.0,9.0,9.0,9.00,198.0,245.0,120.0,187.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9025,S_009026,25,A submission with the following details: Title...,Journal of Applied Economics and Business,NaN,NaN,5.0,4.0,4.0,4.33,...,5.0,4.67,5.0,6.0,6.0,5.67,75.0,50.0,180.0,101.67
9026,S_009027,26,A submission with the following details: Title...,Journal of Applied Economics and Business,NaN,NaN,0.0,1.0,1.0,0.67,...,2.0,2.00,4.0,4.0,3.0,3.67,45.0,75.0,15.0,45.00
9027,S_009028,30,A submission with the following details: Title...,GPT-o1,NaN,NaN,8.0,9.0,8.0,8.33,...,8.0,8.00,8.0,8.0,8.0,8.00,75.0,250.0,200.0,175.00
9028,S_009029,29,A submission with the following details: Title...,GPT-o1,NaN,NaN,8.0,9.0,9.0,8.67,...,8.0,8.00,9.0,8.0,9.0,8.67,75.0,150.0,85.0,103.33


In [7]:
def find_empty_values_in_numbered_columns(df):
    """
    Find empty values in columns ending with _1, _2, _3, or _mean and report their Submission_id.
    
    Args:
        df (pd.DataFrame): Input DataFrame with Submission_id column
        
    Returns:
        dict: Dictionary containing information about empty values
    """
    # Get all columns that end with _1, _2, _3, or _mean
    target_columns = [col for col in df.columns if col.endswith(('_1', '_2', '_3', '_mean'))]
    
    # Dictionary to store results
    empty_values_info = {}
    
    # Check each target column for empty values
    for column in target_columns:
        # Find rows with empty values (None, NaN, or empty string)
        empty_mask = df[column].isna() | (df[column] == '')
        
        if empty_mask.any():
            # Get Submission_ids for rows with empty values
            submission_ids = df.loc[empty_mask, 'Submission_id'].tolist()
            
            # Store information about empty values
            empty_values_info[column] = {
                'count': empty_mask.sum(),
                'submission_ids': submission_ids
            }
    
    return empty_values_info

def print_empty_values_report(empty_values_info):
    """
    Print a formatted report of empty values.
    
    Args:
        empty_values_info (dict): Dictionary containing empty values information
    """
    if not empty_values_info:
        print("No empty values found in columns ending with _1, _2, _3, or _mean")
        return
    
    print("\nEmpty Values Report:")
    print("-" * 80)
    
    for column, info in empty_values_info.items():
        print(f"\nColumn: {column}")
        print(f"Number of empty values: {info['count']}")
        print("Submission IDs with empty values:")
        for submission_id in info['submission_ids']:
            print(f"  - {submission_id}")
    
    print("-" * 80)

# Example usage
try:
    # Run the analysis
    empty_values = find_empty_values_in_numbered_columns(result_df)
    
    # Print the report
    print_empty_values_report(empty_values)
    
    # Optional: Get a quick summary of total empty values per column
    if empty_values:
        print("\nQuick Summary:")
        for column, info in empty_values.items():
            print(f"{column}: {info['count']} empty values")
            
except Exception as e:
    print(f"Error during analysis: {str(e)}")


Empty Values Report:
--------------------------------------------------------------------------------

Column: top5_accept_3
Number of empty values: 1
Submission IDs with empty values:
  - S_008050

Column: top5_accept_rating_criteria_3
Number of empty values: 1
Submission IDs with empty values:
  - S_002577

Column: nobel_potential_3
Number of empty values: 1
Submission IDs with empty values:
  - S_002346

Column: grant_3
Number of empty values: 1
Submission IDs with empty values:
  - S_004716

Column: citation_impact_2
Number of empty values: 1
Submission IDs with empty values:
  - S_008892
--------------------------------------------------------------------------------

Quick Summary:
top5_accept_3: 1 empty values
top5_accept_rating_criteria_3: 1 empty values
nobel_potential_3: 1 empty values
grant_3: 1 empty values
citation_impact_2: 1 empty values


# Replace the gap

In [9]:
def update_ratings(df):
    """
    Interactive function to update ratings and their mean for a specific submission.
    """
    try:
        # 1. Ask for Submission_id
        submission_id = input("Enter the Submission_id: ").strip()
        
        # Verify the Submission_id exists
        if submission_id not in df['Submission_id'].values:
            print(f"Error: Submission_id '{submission_id}' not found in the DataFrame")
            return df
        
        # 2. Ask for the base column name
        base_column = input("Enter the base column name (e.g., 'top5_accept_rating'): ").strip()
        
        # 3. Construct the column names
        columns = [f"{base_column}_{i}" for i in range(1, 4)]
        mean_column = f"{base_column}_mean"
        
        # Verify all columns exist
        all_columns = columns + [mean_column]
        missing_columns = [col for col in all_columns if col not in df.columns]
        if missing_columns:
            print(f"Error: The following columns are missing: {missing_columns}")
            return df
        
        # 4. Get new values for each column
        new_values = []
        for column in columns:
            while True:
                try:
                    value = float(input(f"Enter new value for {column}: "))
                    new_values.append(value)
                    break
                except ValueError:
                    print("Please enter a valid number")
        
        # 5. Calculate mean
        new_mean = sum(new_values) / len(new_values)
        
        # Get the row index
        row_idx = df[df['Submission_id'] == submission_id].index[0]
        
        # 6. Update values
        for column, value in zip(columns, new_values):
            df.at[row_idx, column] = value
        df.at[row_idx, mean_column] = new_mean
        
        # Show the updated values
        print("\nUpdated values:")
        print("-" * 50)
        for column, value in zip(columns, new_values):
            print(f"{column}: {value}")
        print(f"{mean_column}: {new_mean}")
        print("-" * 50)
        
        # Show the actual values from the DataFrame to confirm update
        print("\nConfirming values in DataFrame:")
        print("-" * 50)
        row_data = df.loc[row_idx, all_columns]
        for column in all_columns:
            print(f"{column}: {row_data[column]}")
        print("-" * 50)
        
        return df
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return df

# Example usage:
# result_df = update_ratings(result_df)

In [25]:
result_df = update_ratings(result_df)


Updated values:
--------------------------------------------------
top5_accept_rating_criteria_1: 4.0
top5_accept_rating_criteria_2: 4.0
top5_accept_rating_criteria_3: 4.0
top5_accept_rating_criteria_mean: 4.0
--------------------------------------------------

Confirming values in DataFrame:
--------------------------------------------------
top5_accept_rating_criteria_1: 4.0
top5_accept_rating_criteria_2: 4.0
top5_accept_rating_criteria_3: 4.0
top5_accept_rating_criteria_mean: 4.0
--------------------------------------------------


In [29]:
result_df

,Submission_id,Paper_id,Submission,Original_Publication,Name_Category,Institution,top5_desk_1,top5_desk_2,top5_desk_3,top5_desk_mean,...,nobel_potential_3,nobel_potential_mean,grant_1,grant_2,grant_3,grant_mean,citation_impact_1,citation_impact_2,citation_impact_3,citation_impact_mean
0,S_000001,1,A submission with the following details: Title...,Journal of Political Economy,Top,Massachusetts Institute of Technology;,10.0,9.0,10.0,9.67,...,8.0,8.67,9.0,9.0,8.0,8.67,55.0,150.0,78.0,94.33
1,S_000002,1,A submission with the following details: Title...,Journal of Political Economy,Top,Harvard University;,10.0,10.0,9.0,9.67,...,8.0,9.00,9.0,9.0,9.0,9.00,150.0,250.0,265.0,221.67
2,S_000003,1,A submission with the following details: Title...,Journal of Political Economy,Top,University of Warwick;,10.0,9.0,9.0,9.33,...,9.0,9.00,9.0,9.0,9.0,9.00,120.0,75.0,125.0,106.67
3,S_000004,1,A submission with the following details: Title...,Journal of Political Economy,Top,London School of Economics and Political Science;,9.0,9.0,10.0,9.33,...,8.0,8.33,9.0,10.0,9.0,9.33,85.0,100.0,120.0,101.67
4,S_000005,1,A submission with the following details: Title...,Journal of Political Economy,Top,University of Tokyo;,8.0,9.0,8.0,8.33,...,8.0,8.00,9.0,9.0,9.0,9.00,198.0,245.0,120.0,187.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9025,S_009026,25,A submission with the following details: Title...,Journal of Applied Economics and Business,NaN,NaN,5.0,4.0,4.0,4.33,...,5.0,4.67,5.0,6.0,6.0,5.67,75.0,50.0,180.0,101.67
9026,S_009027,26,A submission with the following details: Title...,Journal of Applied Economics and Business,NaN,NaN,0.0,1.0,1.0,0.67,...,2.0,2.00,4.0,4.0,3.0,3.67,45.0,75.0,15.0,45.00
9027,S_009028,30,A submission with the following details: Title...,GPT-o1,NaN,NaN,8.0,9.0,8.0,8.33,...,8.0,8.00,8.0,8.0,8.0,8.00,75.0,250.0,200.0,175.00
9028,S_009029,29,A submission with the following details: Title...,GPT-o1,NaN,NaN,8.0,9.0,9.0,8.67,...,8.0,8.00,9.0,8.0,9.0,8.67,75.0,150.0,85.0,103.33


# Rerun after filling the gap

In [27]:
# Example usage
try:
    # Run the analysis
    empty_values = find_empty_values_in_numbered_columns(result_df)
    
    # Print the report
    print_empty_values_report(empty_values)
    
    # Optional: Get a quick summary of total empty values per column
    if empty_values:
        print("\nQuick Summary:")
        for column, info in empty_values.items():
            print(f"{column}: {info['count']} empty values")
            
except Exception as e:
    print(f"Error during analysis: {str(e)}")

No empty values found in columns ending with _1, _2, _3, or _mean


In [33]:
result_df.to_csv('/work/final/final_combined_data.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a25c250f-64bb-477e-a263-2c8cc56f7dca' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>